In [22]:
from pathlib import Path
import os

STUDENT = 'mmr497'
DATA_PATH = Path('/local/data/mmr497')
OUTLIERS_PATH = Path('./outliers2/')
INTERMEDIATE_PATH = Path(f'{DATA_PATH}/intermediate_datafiles/')
os.chdir(f'/home/{STUDENT}/')
from util.VisualizeDataset import VisualizeDataset
from Visualiser import Visualiser as Viz
from util.util import ignore_actual_time, read_parquet, write_parquet
from FeatureCreator import FeatureCreatorUpdated
from DataLearningLoader import DataLearningLoader
import pandas as pd
from Chapter7.FeatureSelection import FeatureSelectionClassification
from Chapter7.LearningAlgorithms import ClassificationAlgorithms
from AlteredAlgorithms import AlteredAlgorithmsClassifier
from Chapter7.Evaluation import ClassificationEvaluation
import numpy as np
from CustomPipeline import PreConfiguredPipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Testing parameters
feature_selection = True

In [5]:
EasyViz = Viz()
DataViz = VisualizeDataset('ML2.ipynb')

In [4]:
# cleaned_dataset = read_parquet(INTERMEDIATE_PATH / 'ML4QS_imputed_results.parquet')

In [5]:
# instance_mask = cleaned_dataset.id == 1
# col = 'location_phone_Velocity'
# print(cleaned_dataset.loc[instance_mask, col].isna().sum() / len(cleaned_dataset.loc[instance_mask, col]))
# if cleaned_dataset.loc[instance_mask, col].isna().all():
#     print('All values are NaN for instance {}.'.format(cleaned_dataset.loc[instance_mask, 'id'].values[0]))

In [6]:
# cleaned_dataset.dtypes

In [7]:
# test_df = read_parquet(INTERMEDIATE_PATH / 'non_fourier_features.parquet')
# nan_counts = test_df.isna().sum()
# nan_cols = nan_counts[nan_counts > 0]
# print(nan_cols)

In [8]:
# fcr = FeatureCreatorUpdated(INTERMEDIATE_PATH)
# DLL = DataLearningLoader(df_path=INTERMEDIATE_PATH, output_dir=INTERMEDIATE_PATH, verbose=False)

In [9]:
# feature_df = fcr.create_features(cleaned_dataset, overwrite=True)

In [10]:
# nan_counts = feature_df.isna().sum()
# nan_cols = nan_counts[nan_counts > 0]
# print(nan_cols)

In [11]:
# # Stupid fix! Only temporary! MUST REMOVE
# def remove_nan_cols(df):
#     df_fs = df.copy()
#     col_before = df_fs.columns
#     nan_counts = df_fs.isna().sum()
#     nan_cols = nan_counts[nan_counts > 0]
#     df_fs = df_fs.drop(nan_cols.index, axis=1)
#     col_after = df_fs.columns
#     print('Removed columns: ', [col for col in col_before if col not in col_after])
#     return df_fs
#
# filtered_feature_df = remove_nan_cols(feature_df)

In [12]:
# X_train, X_test, y_train, y_test = DLL.prepare_data(filtered_feature_df)

In [6]:
intermediate_df = read_parquet(INTERMEDIATE_PATH / 'ML4QS_combined_results_example.parquet')

In [7]:
intermediate_df.dtypes

id                                      int64
acc_phone_X                           float64
acc_phone_Y                           float64
acc_phone_Z                           float64
lin_acc_phone_X                       float64
lin_acc_phone_Y                       float64
lin_acc_phone_Z                       float64
gyr_phone_X                           float64
gyr_phone_Y                           float64
gyr_phone_Z                           float64
location_phone_Latitude               float64
location_phone_Longitude              float64
location_phone_Height                 float64
location_phone_Velocity               float64
location_phone_Direction              float64
location_phone_Horizontal Accuracy    float64
location_phone_Vertical Accuracy      float64
mag_phone_X                           float64
mag_phone_Y                           float64
mag_phone_Z                           float64
proximity_phone_Distance              float64
labeltram                         

In [8]:
def write_splitted_data(data):

    if len(data[0]) == 1:
        print("Expecting format: (data, name) ... continuing.")

    for data, name in data:
        try:
            write_parquet(data, INTERMEDIATE_PATH / f'{name}_postpipe.parquet')
        except:
            print("Can't write parquet file for ", name, " ... continuing.")

In [10]:
pipe_NF = PreConfiguredPipeline(intermediate_path=INTERMEDIATE_PATH, include_fourier=False)
X_train, X_test, y_train, y_test = pipe_NF.fit_transform(intermediate_df, verbose=True, overwrite=True, pipe_name="NF")

[INFO] Removing bad sensors...
[INFO] Removing bad sensors...
[INFO] Splitting data into train and test sets...
[INFO] Splitting data into train and test sets...


WARNING THIS FEATURE CREATOR USE NO FOURIER FEATURES


[INFO] Starting preprocessing pipeline...
[INFO] Starting preprocessing pipeline...
[INFO] Preprocessing NF_X_train...
[INFO] Preprocessing NF_X_train...
[INFO] Checking for outliers...
[INFO] Checking for outliers...


bike: Only 2 sessions - 1 train, 1 test


[INFO] Ensuring correct data types...
[INFO] Ensuring correct data types...
[INFO] Imputing missing values...
[INFO] Imputing missing values...


...

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/NF_X_train.parquet 

Cleaning data...
Cleaning features...


[INFO] Ensuring correct data types...
[INFO] Ensuring correct data types...
[INFO] Imputing missing values...
[INFO] Imputing missing values...
[INFO] Creating features...
[INFO] Creating features...


No observed values for location_phone_Velocity for vehicle type labeltrain.
No observed values for location_phone_Direction for vehicle type labeltrain.


[INFO] Dropping any direction data!
[INFO] Dropping any direction data!
[INFO] Cleaning data...
[INFO] Cleaning data...
[INFO] Preprocessing NF_X_test complete!
[INFO] Preprocessing NF_X_test complete!


Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/NF_X_test.parquet 

Cleaning data...
Cleaning features...


In [11]:
nf_data = [(X_train, "X_train_NF"), (X_test, "X_test_NF"), (y_train, "y_train_NF"), (y_test, "y_test_NF")]
write_splitted_data(nf_data)

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/X_train_NF_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/X_test_NF_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/y_train_NF_postpipe.parquet 

Successfully written to parquet file at  /local/data/mmr497/intermediate_datafiles/y_test_NF_postpipe.parquet 



In [12]:
pipe = PreConfiguredPipeline(intermediate_path=INTERMEDIATE_PATH)
X_train, X_test, y_train, y_test = pipe.fit_transform(intermediate_df, verbose=True, overwrite=False)

[INFO] Removing bad sensors...
[INFO] Removing bad sensors...
[INFO] Removing bad sensors...
[INFO] Splitting data into train and test sets...
[INFO] Splitting data into train and test sets...
[INFO] Splitting data into train and test sets...
[INFO] Starting preprocessing pipeline...
[INFO] Starting preprocessing pipeline...
[INFO] Starting preprocessing pipeline...
[INFO] Preprocessing X_train...
[INFO] Preprocessing X_train...
[INFO] Preprocessing X_train...
[INFO] Checking for outliers...
[INFO] Checking for outliers...
[INFO] Checking for outliers...


bike: Only 2 sessions - 1 train, 1 test


[INFO] Ensuring correct data types...
[INFO] Ensuring correct data types...
[INFO] Ensuring correct data types...
[INFO] Imputing missing values...
[INFO] Imputing missing values...
[INFO] Imputing missing values...


No observed values for location_phone_Direction for vehicle type labeltrain.
No observed values for location_phone_Direction for vehicle type labeltrain.
No observed values for location_phone_Direction for vehicle type labeltrain.
No observed values for location_phone_Direction for vehicle type labeltrain.


[INFO] Creating features...
[INFO] Creating features...
[INFO] Creating features...
[INFO] Dropping any direction data!
[INFO] Dropping any direction data!
[INFO] Dropping any direction data!


...

[INFO] Ensuring correct data types...
[INFO] Ensuring correct data types...
[INFO] Ensuring correct data types...
[INFO] Imputing missing values...
[INFO] Imputing missing values...
[INFO] Imputing missing values...
[INFO] Creating features...
[INFO] Creating features...
[INFO] Creating features...


No observed values for location_phone_Velocity for vehicle type labeltrain.
No observed values for location_phone_Direction for vehicle type labeltrain.
Combined features already exist at /local/data/mmr497/intermediate_datafiles/X_test_combined_features.parquet


[INFO] Dropping any direction data!
[INFO] Dropping any direction data!
[INFO] Dropping any direction data!
[INFO] Cleaning data...
[INFO] Cleaning data...
[INFO] Cleaning data...


Loaded combined features from /local/data/mmr497/intermediate_datafiles/X_test_combined_features.parquet
If this was not intended, rerun create_features with overwrite=True
Cleaning data...
Cleaning features...


[INFO] Preprocessing X_test complete!
[INFO] Preprocessing X_test complete!
[INFO] Preprocessing X_test complete!


In [ ]:
# pipe = PreConfiguredPipeline(intermediate_path=INTERMEDIATE_PATH)
# X_train, X_test, y_train, y_test = pipe.fit_transform(intermediate_df, verbose=True, overwrite=False)

In [18]:
y_test

timestamp
1970-01-01 00:54:19.750    metro
1970-01-01 00:54:20.000    metro
1970-01-01 00:54:20.250    metro
1970-01-01 00:54:20.500    metro
1970-01-01 00:54:20.750    metro
                           ...  
1970-01-01 03:30:08.000    train
1970-01-01 03:30:08.250    train
1970-01-01 03:30:08.500    train
1970-01-01 03:30:08.750    train
1970-01-01 03:30:09.000    train
Name: transport_mode, Length: 20479, dtype: object

In [ ]:
# write_splitted_data([(X_train, "X_train"), (X_test, "X_test"), (y_train, "y_train"), (y_test, "y_test")])

In [19]:
# Optional for testing

def remove_problem_cols(df):
    # This is for testing purposes, stupid fix!
    df_fs = df.copy()
    col_before = df_fs.columns
    df_fs = df_fs.select_dtypes(include=[np.number], exclude=['timedelta64[ns]'])
    col_after = df_fs.columns
    print('Removed columns: ', [col for col in col_before if col not in col_after])
    return df_fs

def feature_selection(X_train, X_test, y_train, y_test):
    fs = FeatureSelectionClassification()

    N_FORWARD_SELECTION = 50

    X_train_fs = remove_problem_cols(X_train)
    X_test_fs = remove_problem_cols(X_test)

    features, ordered_features, ordered_scores = fs.forward_selection(N_FORWARD_SELECTION,
                                                                          X_train_fs,
                                                                          X_test_fs,
                                                                          y_train,
                                                                          y_test,
                                                                          gridsearch=True)

    return features, ordered_features, ordered_scores

        # DataViz.plot_xy(x=[range(1, N_FORWARD_SELECTION+1)], y=[ordered_scores],
        #             xlabel='number of features', ylabel='accuracy')

In [20]:
class EvaluationContainer:
    def __init__(self, model_name):
        self.model_name = model_name

In [26]:
# nan_counts = X_test_fs.isna().sum()
# nan_cols = nan_counts[nan_counts > 0]
# print(nan_cols)

location_phone_Velocity           4522
location_phone_Velocity_mean      4522
location_phone_Velocity_median    4522
location_phone_Velocity_min       4522
location_phone_Velocity_max       4522
session_avg_velocity              4522
session_velocity_std              4522
dtype: int64


In [47]:
def plot_class_distribution(data, title: str = 'Class distribution'):
    if isinstance(data, pd.DataFrame):
        data['transport_mode'] = data['transport_mode'].astype('string')
        print(data['transport_mode'].dtypes)
        sns.countplot(x='transport_mode', data=data)
        plt.xlabel('transport_mode')
        plt.ylabel('Count')
        plt.title(title)
        plt.show()
    elif isinstance(data, pd.Series):
        data = pd.Series(data)
        sns.countplot(x=data)
        plt.xlabel('Label')
        plt.ylabel('Count')
        plt.title(title)
        plt.show()

In [41]:
y_test_test = y_test.reset_index()

In [39]:
y_test_test['transport_mode'].dtypes

dtype('O')

In [48]:
plot_class_distribution(y_test_test)

string
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/local/data/mmr497/.conda/envs/ML4QS/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_685906/105364256.py", line 1, in <module>
    plot_class_distribution(y_test_test)
  File "/tmp/ipykernel_685906/3193065897.py", line 5, in plot_class_distribution
    sns.countplot(x='transport_mode', data=data)
  File "/local/data/mmr497/.conda/envs/ML4QS/lib/python3.8/site-packages/seaborn/categorical.py", line 2631, in countplot
    p = _CategoricalAggPlotter(
  File "/local/data/mmr497/.conda/envs/ML4QS/lib/python3.8/site-packages/seaborn/categorical.py", line 67, in __init__
    super().__init__(data=data, variables=variables)
  File "/local/data/mmr497/.conda/envs/ML4QS/lib/python3.8/site-packages/seaborn/_base.py", line 634, in __init__
    self.assign_variables(data, variables)
  File "/local/data/mmr497/.conda/envs/ML4QS/lib/python3.8/si

In [27]:
# Setup model
CA = ClassificationAlgorithms()
alt_CA = AlteredAlgorithmsClassifier()

X_train_fs = remove_problem_cols(X_train)
X_test_fs = remove_problem_cols(X_test)

# TODO: Delete after testing

test_X_test = X_test_fs.copy()
test_X_test = test_X_test.fillna(0)

nb_train_y, nb_test_y, nb_train_prob_y, nb_test_prob_y = alt_CA.naive_bayes(
        X_train_fs, y_train, test_X_test
    )


Removed columns:  ['original_time', 'time_diff', 'shifted_time']
Removed columns:  ['original_time', 'time_diff', 'shifted_time']


In [29]:
# Evaluate models
evalCA = ClassificationEvaluation()

evaluations = {}

print(classification_report(y_test, nb_test_y, zero_division=0))
print(confusion_matrix(y_test, nb_test_y))

              precision    recall  f1-score   support

        bike       0.00      0.00      0.00      2423
         bus       0.00      0.00      0.00      1915
         car       0.00      0.00      0.00      4136
       metro       0.33      0.17      0.23      3061
       train       0.35      1.00      0.52      4522
        tram       0.00      0.00      0.00       808
     walking       0.60      1.00      0.75      3614

    accuracy                           0.42     20479
   macro avg       0.18      0.31      0.21     20479
weighted avg       0.23      0.42      0.28     20479

[[   0    0    0    0    0    0 2423]
 [   0    0    0    0 1915    0    0]
 [   0    0    0 1082 3054    0    0]
 [   0    0    0  533 2528    0    0]
 [   0    0    0    0 4522    0    0]
 [   0    0    0    0  808    0    0]
 [   0    0    0    0    0    0 3614]]
